In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from konlpy.tag import Okt   

In [3]:
res_cer = pd.read_csv("resume_certificate.csv")
len(res_cer['resume_seq'].unique())
# 자격증 소유자 5976명

5976

In [4]:
len(res_cer['certificate_contents'].unique())
# 자격증 종류 1787

1787

In [5]:
res_cer['certificate_contents'].astype(str)
useful = ['1종', '2종', '2급', '1급', '3급', '1단', '3단', '2단', '4종', '4단', '4급', '6시', '6급', '1차', '5급', '7급', '3보', '3D','기능사','산업기사','기사','기능장','산업 기사','기술사']
pattern = '|'.join(useful)
res_cer['classic'] = res_cer['certificate_contents'].str.extract(f'({pattern})')
print(res_cer['classic'])


for i in range(len(res_cer)):
    for j in useful:
        if isinstance(res_cer['certificate_contents'][i], str):  # 문자열인지 확인
            res_cer['certificate_contents'][i] = res_cer['certificate_contents'][i].replace(j, "")

0         NaN
1          2종
2          2종
3          2종
4          3급
         ... 
12970    산업기사
12971     기능사
12972     기능사
12973      2급
12974      2급
Name: classic, Length: 12975, dtype: object


In [6]:
res_cer['certificate_contents']
# 자격증에서 등급을 제거하여 classic에 담고 기존 자격증 칸의 등급을 없애서 문자처리를 원할하게 설정

0              손해보험사
1              운전면허증
2        자동차운전면허  보통
3             운전면허보통
4               메이크업
            ...     
12970          컬러리스트
12971      컴퓨터그래픽스운용
12972         수치제어밀링
12973             주산
12974         유통관리사 
Name: certificate_contents, Length: 12975, dtype: object

In [9]:
okt = Okt()  
set_index = []
for i in range(len(res_cer)):
    if isinstance(res_cer['certificate_contents'][i], str): 
        setting = okt.nouns(res_cer['certificate_contents'][i])
        set_index.extend(setting)
check = set_index
check # 길이가 1개짜리인 글자는 제거. 

['손해',
 '보험사',
 '운전면허증',
 '자동차',
 '운전면허',
 '보통',
 '운전면허',
 '보통',
 '메이크업',
 '파워포인트',
 '파워포인트',
 '패션',
 '머',
 '이징',
 '자동차',
 '운전면허',
 '보통',
 '파워포인트',
 '포토샵',
 '운전면허',
 '보통',
 '자동차',
 '운전면허',
 '보통',
 '전산',
 '회계',
 '운용',
 '사',
 '운전면허',
 '보통',
 '컴퓨터',
 '활용',
 '능력',
 '교원',
 '자격증',
 '실기',
 '교사',
 '한재',
 '컬러',
 '리스트',
 '정보기기',
 '운영',
 '운전면허',
 '보통',
 '자동차',
 '운전면허',
 '보통',
 '운전면허증',
 '자동차',
 '운전면허',
 '보통',
 '미용',
 '자동차',
 '운전면허증',
 '중등',
 '교사',
 '운전면허',
 '보통',
 '보통',
 '운전면허증',
 '운전면허',
 '보통',
 '운전면허',
 '보통',
 '자동차',
 '운전면허',
 '보통',
 '컴퓨터그래픽스',
 '운용',
 '자동차',
 '운전면허',
 '보통',
 '워드프로세서',
 '컴퓨터그래픽스',
 '운용',
 '실용',
 '영어',
 '자동차',
 '운전면허',
 '자동',
 '국가',
 '공인',
 '정보기술',
 '자동차',
 '정비',
 '컬러',
 '리스트',
 '자동차',
 '운전면허',
 '보통',
 '칼라',
 '리스트',
 '비주',
 '얼머',
 '저',
 '유통',
 '관리사',
 '샵마스터',
 '포토샵',
 '컬러',
 '리스트',
 '한자',
 '자격',
 '검정',
 '문서실무사',
 '워드프로세서',
 '일러스트',
 '자동차',
 '운전면허',
 '보통',
 '정보기기',
 '운영',
 '정보기술',
 '자격',
 '파워포인트',
 '정보통',
 '운용',
 '자동차',
 '운전면허',
 '보통',
 '운전면허증',
 '보통',
 '샵마스터',
 '한글',
 '엑셀',
 '컴퓨

In [10]:
check_vc = pd.Series(check).value_counts()
check_vc2 = []
for i in check_vc.index:
    if len(i) > 1:
        check_vc2.append(i)

In [11]:
len(check_vc2)     # 자격증 카테고리화

735

In [12]:
for k in check_vc2:
    res_cer[f'{k}'] = 0  # 초기값 설정. 자격증 단어마다 인덱스를 제작

C:\Users\user\AppData\Local\Temp\ipykernel_6504\2336250081.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  res_cer[f'{k}'] = 0  # 초기값 설정
C:\Users\user\AppData\Local\Temp\ipykernel_6504\2336250081.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  res_cer[f'{k}'] = 0  # 초기값 설정
C:\Users\user\AppData\Local\Temp\ipykernel_6504\2336250081.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

C:\Users\user\AppData\Local\Temp\ipykernel_6504\2336250081.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  res_cer[f'{k}'] = 0  # 초기값 설정
C:\Users\user\AppData\Local\Temp\ipykernel_6504\2336250081.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  res_cer[f'{k}'] = 0  # 초기값 설정
C:\Users\user\AppData\Local\Temp\ipykernel_6504\2336250081.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

In [13]:
for i in range(len(res_cer)):
    j = res_cer['certificate_contents'][i]
    for k in range(len(check_vc2)):
        if isinstance(k, str):
            if re.search(check_vc2[k], j) is not None :
                res_mer[f'{k}'][i] = 1                     # 자격증 get_dummies한 것과 비슷한 효과

In [14]:
res_cer

,resume_seq,certificate_contents,classic,보통,운전면허,자동차,운전면허증,리스트,워드프로세서,컬러,...,화학공학,어도비,캐드,중등교육,기획사,컨벤션,종이접기,측량,훈육,유기농업
0,U06421,손해보험사,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,U01278,운전면허증,2종,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,U01008,자동차운전면허 보통,2종,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,U03248,운전면허보통,2종,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,U07483,메이크업,3급,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12970,U07453,컬러리스트,산업기사,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12971,U06358,컴퓨터그래픽스운용,기능사,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12972,U03248,수치제어밀링,기능사,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12973,U08157,주산,2급,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
res_cer['classic']=res_cer['classic'].astype('category')  # 자격증 등급을 카테고리화하여 get_dummies
jc = pd.get_dummies(res_cer['classic'],prefix ='job_class',dtype='int') 
res_cer = pd.concat([res_cer,jc],axis=1)
res_cer = res_cer.drop(['certificate_contents','classic'],axis=1) # 자격증과 분류 제거

In [16]:
res_cer = res_cer.set_index('resume_seq').sort_index()
res_cer 

,보통,운전면허,자동차,운전면허증,리스트,워드프로세서,컬러,컴퓨터,능력,관리사,...,job_class_4종,job_class_5급,job_class_6급,job_class_6시,job_class_7급,job_class_기능사,job_class_기능장,job_class_기사,job_class_산업 기사,job_class_산업기사
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
U00003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
U08480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U08481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
res_cer = res_cer.groupby('resume_seq').agg('sum')
res_cer # 유저별로 합쳐서 더하기

,보통,운전면허,자동차,운전면허증,리스트,워드프로세서,컬러,컴퓨터,능력,관리사,...,job_class_4종,job_class_5급,job_class_6급,job_class_6시,job_class_7급,job_class_기능사,job_class_기능장,job_class_기사,job_class_산업 기사,job_class_산업기사
resume_seq,,,,,,,,,,,,,,,,,,,,,
U00002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
U00004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
U00006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U00007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
U08476,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U08479,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
U08480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
